In [24]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import seaborn as sns
import csv
from scipy.io import arff
from sklearn import datasets
from sklearn.cluster import KMeans
import numpy as np
from kmodes.kmodes import KModes
from kmodes.util.dissim import ng_dissim


In [139]:
#data = arff.loadarff('universityPolicyTrainingData.arff')
data = pd.read_csv('permittedtest.csv')
print(data)
df = pd.DataFrame(data)
df= df.drop(columns ='Unnamed: 9')

print()
print(df.describe())
df.head()

    odepartment        action position         type  ischair    crs crstaught  \
0            cs   changescore  faculty    gradebook     True  cs101     cs601   
1            cs          read  student   transcript    False  cs101     ee601   
2            cs          read  faculty   transcript     True  cs101     ee601   
3            cs          read  faculty    gradebook     True  cs601     cs601   
4            cs   changescore  faculty    gradebook     True  cs101     cs601   
5            cs          read  student   transcript     True  cs101     cs602   
6            ee          read  student   transcript     True  cs101     ee601   
7            cs          read  faculty   transcript    False  cs602     cs602   
8            ee          read  faculty    gradebook     True  cs602     cs602   
9            cs   changescore  faculty    gradebook     True  cs602     cs601   
10           cs          read  faculty   transcript    False  cs601     cs602   
11           cs     setstatu

,odepartment,action,position,type,ischair,crs,crstaught,udepartment,crstaken
0,cs,changescore,faculty,gradebook,True,cs101,cs601,admissions,cs601
1,cs,read,student,transcript,False,cs101,ee601,registrar,cs601
2,cs,read,faculty,transcript,True,cs101,ee601,registrar,cs101
3,cs,read,faculty,gradebook,True,cs601,cs601,registrar,cs101
4,cs,changescore,faculty,gradebook,True,cs101,cs601,registrar,cs101


In [141]:
#print(df.describe())

df.isnull().values.any()
df["position"] = df["position"].astype('category')
df["udepartment"] = df["udepartment"].astype('category')
df["crstaken"] = df["crstaken"].astype('category')
df["crstaught"] = df["crstaught"].astype('category')
df["ischair"] = df["ischair"].astype('category')
df["type"] = df["type"].astype('category')
df["crs"] = df["crs"].astype('category')
df["odepartment"] = df["odepartment"].astype('category')
df["action"] = df["action"].astype('category')



df.head(30)
print(df.dtypes)

odepartment    category
action         category
position       category
type           category
ischair        category
crs            category
crstaught      category
udepartment    category
crstaken       category
dtype: object


In [112]:
# Attributes Frequencies
datafreq = pd.DataFrame()
for col in df.columns:
    datafreq=df[col].value_counts()
    #print (datafreq)
    
#print(datafreq)

In [142]:
# Correlation Function

def cramers_v(confusion_matrix):
    """ calculate Cramers V statistic for categorial-categorial association.
        uses correction from Bergsma and Wicher,
        Journal of the Korean Statistical Society 42 (2013): 323-328
    """
    chi2 = ss.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

# Writing Data Correlation in a CSV
with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
    
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()



/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [143]:
data = df

km = KModes(n_clusters=12, init='Cao', n_init=1, verbose=2)
df_dummy = pd.get_dummies(df)

clusters = km.fit_predict(data)
df_dummy['clusters'] = clusters

# Print the cluster centroids
print(km.cluster_centroids_)

Init: initializing centroids
Init: initializing clusters
Starting iterations...
Run 1, iteration: 1/100, moves: 20, cost: 309.0
Run 1, iteration: 2/100, moves: 1, cost: 309.0
[['cs' 'read' 'faculty' 'transcript' 'True' 'cs101' 'ee601' 'registrar'
  'cs101']
 ['ee' 'read' 'student' 'transcript' 'True' 'cs101' 'cs601' 'cs' 'cs601']
 ['ee' 'read' 'student' 'transcript' 'False' 'cs602' 'cs602' 'registrar'
  'cs101']
 ['cs' 'read' 'staff' 'transcript' 'False' 'cs101' 'cs602' 'cs' 'cs601']
 ['cs' 'read' 'faculty' 'gradebook' 'False' 'cs601' 'cs601' 'registrar'
  'cs601']
 ['cs' 'read' 'faculty' 'transcript' 'False' 'cs602' 'cs601' 'cs' 'cs101']
 ['ee' 'read' 'faculty' 'transcript' 'False' 'cs602' 'ee601' 'admissions'
  'cs601']
 ['cs' 'checkstatus' 'student' 'application' 'True' 'cs602' 'cs602' 'cs'
  'cs601']
 ['cs' 'setstatus' 'faculty' 'application' 'True' 'cs601' 'cs601'
  'admissions' 'cs101']
 ['cs' 'readmyscores' 'faculty' 'gradebook' 'False' 'cs101' 'cs602'
  'admissions' 'cs101']
 [

In [144]:
cluster_map = pd.DataFrame()
cluster_map['data_index'] = data.index.values
cluster_map['cluster'] = km.labels_

#valuescount=pd.DataFrame()
#temp=[]

for i in range(0, 9):
    ind=cluster_map.cluster == i
    #print(ind)

In [145]:
cluster_map
ndf=pd.concat([data,cluster_map], axis=1)
ndf=ndf.drop(columns='data_index')
ndf

,odepartment,action,position,type,ischair,crs,crstaught,udepartment,crstaken,cluster
0,cs,changescore,faculty,gradebook,True,cs101,cs601,admissions,cs601,11
1,cs,read,student,transcript,False,cs101,ee601,registrar,cs601,0
2,cs,read,faculty,transcript,True,cs101,ee601,registrar,cs101,0
3,cs,read,faculty,gradebook,True,cs601,cs601,registrar,cs101,4
4,cs,changescore,faculty,gradebook,True,cs101,cs601,registrar,cs101,11
5,cs,read,student,transcript,True,cs101,cs602,cs,cs601,1
6,ee,read,student,transcript,True,cs101,ee601,registrar,cs601,1
7,cs,read,faculty,transcript,False,cs602,cs602,registrar,cs101,2
8,ee,read,faculty,gradebook,True,cs602,cs602,cs,cs101,10
9,cs,changescore,faculty,gradebook,True,cs602,cs601,cs,cs601,11


In [146]:
# Calculating Attributes Effectevness in each cluster
temp = pd.DataFrame()
datafreq = pd.DataFrame()
print(datafreq)
clusTemp = pd.DataFrame()
for i in range(0,13):
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    for col in df.columns:
        datafreq = df[col].value_counts().to_dict()
        #datafreq= pd.DataFrame.from_dict(datafreq, )
        temp=clusTemp[col].value_counts().to_dict()
        #temp= pd.DataFrame.from_dict(temp)
        #print(temp)
        #print(datafreq)
        for key, value in datafreq.items():
        #    datafreq["count"] = df.loc[col == attr].size()
            #print(key)
            x = value 
            #print(x)
            for key1, value1 in temp.items():
                if key == key1:
                    y = value1
                    #print(y)
                    if y/x >= 0.15:
                        print('the effectevniess of ', key, 'is: ', y/x)

  


Empty DataFrame
Columns: []
Index: []
CLUSTER NUMBER 0
the effectevniess of  cs is:  0.39215686274509803
the effectevniess of  ee is:  0.24528301886792453
the effectevniess of  read is:  0.3643410852713178
the effectevniess of  write is:  0.8
the effectevniess of  readmyscores is:  0.2
the effectevniess of  faculty is:  0.2894736842105263
the effectevniess of  student is:  0.3508771929824561
the effectevniess of  staff is:  0.5
the effectevniess of  transcript is:  0.36283185840707965
the effectevniess of  gradebook is:  0.17391304347826086
the effectevniess of  application is:  0.2857142857142857
the effectevniess of  roster is:  0.8
the effectevniess of  True is:  0.41
the effectevniess of  False is:  0.21818181818181817
the effectevniess of  cs101 is:  0.5064935064935064
the effectevniess of  cs602 is:  0.17777777777777778
the effectevniess of  cs601 is:  0.18181818181818182
the effectevniess of  ee601 is:  0.625
the effectevniess of  cs601 is:  0.21428571428571427
the effectevniess

In [147]:
datacorr = pd.DataFrame()
for i in range(0,13):
    print("CLUSTER NUMBER", i)
    clusTemp = ndf.loc[ndf['cluster'] == i]
    for col in df.columns:

    #writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            datacorr = cramers_v(confusion_matrix)
            temp=clusTemp[col].value_counts().to_dict()    
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            if temp >= 0.5:
                print('correlation between ',col,'and', col2,'is : ', temp )
                

CLUSTER NUMBER 0
correlation between  odepartment and odepartment is :  0.9855696921908789


/Users/aldairi/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  if __name__ == '__main__':


correlation between  action and action is :  1.0
correlation between  action and type is :  0.8083735292276871
correlation between  position and position is :  0.9999999999999999
correlation between  type and action is :  0.8083735292276871
correlation between  type and type is :  1.0
correlation between  ischair and ischair is :  0.9858163309581752
correlation between  crs and crs is :  1.0
correlation between  crstaught and crstaught is :  0.9999999999999999
correlation between  udepartment and udepartment is :  0.9999999999999999
correlation between  crstaken and crstaken is :  0.9851408845979768
CLUSTER NUMBER 1
correlation between  odepartment and odepartment is :  0.9855696921908789
correlation between  action and action is :  1.0
correlation between  action and type is :  0.8083735292276871
correlation between  position and position is :  0.9999999999999999
correlation between  type and action is :  0.8083735292276871
correlation between  type and type is :  1.0
correlation betw

correlation between  odepartment and odepartment is :  0.9855696921908789
correlation between  action and action is :  1.0
correlation between  action and type is :  0.8083735292276871
correlation between  position and position is :  0.9999999999999999
correlation between  type and action is :  0.8083735292276871
correlation between  type and type is :  1.0
correlation between  ischair and ischair is :  0.9858163309581752
correlation between  crs and crs is :  1.0
correlation between  crstaught and crstaught is :  0.9999999999999999
correlation between  udepartment and udepartment is :  0.9999999999999999
correlation between  crstaken and crstaken is :  0.9851408845979768


In [ ]:
# Correlation in Each Cluster

with open("dcorr.csv",'w',newline='') as f:
    for col in df.columns:
        
        writer = csv.writer(f, dialect='excel')
        for col2 in df.columns:
            
            confusion_matrix = pd.crosstab(df[col], df[col2]).as_matrix()
            temp = cramers_v(confusion_matrix)
            writer.writerow({temp, col2 , col })
    f.close()

